In [2]:
import numpy as np
import pandas as pd
# import shapefile as shp
# import matplotlib.pyplot as plt
# import seaborn as sns
import geopandas as gpd
from shapely.geometry import MultiPolygon,Polygon, LineString, Point
from collections import defaultdict

In [3]:
data = gpd.read_file('kommuner.geojson')
# https://github.com/Neogeografen/dagi/tree/master/geojson

In [4]:
results = pd.read_csv('../../export_okt1_22/WCA_export_Results.tsv',sep='\t')
comps = pd.read_csv('../../export_okt1_22/WCA_export_Competitions.tsv',sep='\t')

In [5]:
comps = comps[comps['countryId']=='Denmark']
comps.rename(columns={'id':'competitionId'},inplace=True)
dk_res = comps.merge(results,left_on='competitionId',right_on='competitionId')

In [6]:
def getCompAndCords():
    personToComps = defaultdict(set)
    compCords = {}
    for val in dk_res.values:
        personToComps[val[27]].add(val[0])
        compCords[val[0]] = (val[19]/10e5,val[20]/10e5)
    return compCords, personToComps

In [7]:
def getKommunerByPerson(personToComps,compCords):
    komcount = defaultdict(set) # Person -> set
    for per in personToComps:
        for comp in personToComps[per]:
            x,y = compCords[comp]
            for idx,val in enumerate(data.geometry):
                if Polygon(val).contains(Point(y,x)):
                    komcount[per].add(data.KOMNAVN[idx])
                    break
    return komcount

In [8]:
compCords, personToComps = getCompAndCords()

In [9]:
komcount = getKommunerByPerson(personToComps,compCords)

In [10]:
overview = [(per,len(komcount[per])) for per in komcount]
overview.sort(key=lambda x:x[1],reverse=True)

In [11]:
for val in overview[:20]:
    print(val) # Print top 20

('2011TRON01', 29)
('2006BUUS01', 28)
('2008ANDE02', 26)
('2013EGDA02', 24)
('2013EGDA01', 24)
('2015KANS01', 23)
('2010BECK01', 22)
('2015JORG01', 22)
('2012GOOD02', 21)
('2008NIEL01', 21)
('2006BUUS02', 20)
('2014DETL01', 20)
('2017JARN01', 19)
('2010MOFF02', 19)
('2018KJOL01', 17)
('2010NIEL01', 16)
('2017BYRN02', 16)
('2017BYRN01', 16)
('2017EXNE01', 16)
('2013NIEL02', 16)


In [12]:
def getCompsPerKommune(compCords):
    compsPerKommune = defaultdict(int) # name -> count
    for comp in compCords:
        x,y = compCords[comp]
        for idx,val in enumerate(data.geometry):
            if Polygon(val).contains(Point(y,x)):
                compsPerKommune[data.KOMNAVN[idx]] += 1
                break
    return compsPerKommune

In [13]:
compsPerKommune = getCompsPerKommune(compCords)

In [14]:
overviewK = [(per,compsPerKommune[per]) for per in compsPerKommune]
overviewK.sort(key=lambda x:x[1],reverse=True)

In [15]:
overviewK

[('Roskilde', 12),
 ('Tårnby', 9),
 ('Ballerup', 8),
 ('Odense', 8),
 ('Hvidovre', 7),
 ('Aarhus', 5),
 ('Lejre', 5),
 ('Svendborg', 5),
 ('København', 4),
 ('Faaborg-Midtfyn', 4),
 ('Lyngby-Taarbæk', 3),
 ('Slagelse', 3),
 ('Frederikshavn', 3),
 ('Furesø', 2),
 ('Aalborg', 1),
 ('Assens', 1),
 ('Nyborg', 1),
 ('Vejle', 1),
 ('Kolding', 1),
 ('Hillerød', 1),
 ('Allerød', 1),
 ('Brøndby', 1),
 ('Fredensborg', 1),
 ('Guldborgsund', 1),
 ('Fredericia', 1),
 ('Greve', 1),
 ('Frederiksberg', 1),
 ('Odsherred', 1),
 ('Rebild', 1),
 ('Ringsted', 1),
 ('Skanderborg', 1),
 ('Haderslev', 1),
 ('Varde', 1),
 ('Holstebro', 1)]